In [ ]:
# Example 4.2: Jack's Car Rental from chapter 4: Dynamic programming of Sutton and Barto’s book: Reinforcement Learning: An Introduction
# car inventory management using dynamic programming. 
#The demand and suppy of the inventory is assumed to have a poisson distribution 
# the enviornment dynamic are decided as per Markov Decision Process 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import *

In [15]:
# at loc1 on an average in one day 3 cars are requested and 3 are returned and for loc2 4 are requested and 2 are returned
expected_request_loc1 = 3
expected_return_loc1 = 3
expected_request_loc2 = 4 
expected_return_loc2 = 2
# other variables (assumed)
max_cars = 20
max_moved = 5 
discount_rate = 0.9

In [14]:
# states (number of cars at each of the two locations at a given time)
states=[]
for i in range(max_cars+1):
    for j in range(max_cars+1):
        states.append([i,j])

In [13]:
#policy # initialised to zero i.e probability of any action from a given state is zero. hence no action is taken. No cars moved 
# deterministic policy 
policy = np.zeros((max_cars + 1, max_cars + 1))

In [5]:
# actions # positive for loc1=>loc2 and negative for loc22=>loc1
actions = np.arange(-max_moved,max_moved+1)

In [6]:
# Reward
credit = 10
loss = -2 

In [7]:
# Poisson distribution 
# Cars requested or returned at each location on a given day
# probability that there are n customers or n returns on a given day
#p(n) = ((lamda)^n/n!)*e^(-lamda)  #n = 0,1,2,3,4,5,6,7,8,9,10 as expected value(lamda) lie around 3 & 4 hence probability of n> 10 is very low
def poisson(lam):
    temp ={}
    for key in range(11):
        temp[key] = (np.exp(-lam) * pow(lam, key)) / factorial(key)
    return temp

In [8]:
cars_requested_loc1 = poisson(expected_request_loc1)
cars_returned_loc1 = poisson(expected_return_loc1)
cars_requested_loc2 = poisson(expected_request_loc2)
cars_returned_loc2 = poisson(expected_return_loc2)

In [9]:
#value function
currentStateVal = np.zeros((max_cars + 1, max_cars + 1))
newStateVal = np.zeros((max_cars + 1, max_cars + 1))

In [10]:
# reward +10$ for fulfilling a request and -2$ loss for every unfulfilled request
def reward(state,cars_requested):
    if state >= cars_requested:
        reward = 10
    else:
        reward = -2 
    return reward    

In [43]:
# policy evaluation
#state_start_2 = state_end_1 + actions + cars_returned_1
#state_end_1 = state_start_1 - cars_requested_1
improvePolicy = False

for i in range(len(states)):
    newStateVal[states[i][0], states[i][1]] = new_values(states[i], policy[states[i][0], states[i][1]], currentStateVal)

In [47]:
# stop improving state value function if the difference between new value and current value is small 
if np.sum(np.absolute(newStateVal - currentStateVal)) < 1e-4:
    currentStateVal = newStateVal.copy()
    improvePolicy = True

In [48]:
improvePolicy

True

In [11]:
# iterate over all cases of returns and requests to get net probability of next states for a given state and action, p(s',r|s,r) 
def new_values(current_state, action, currentStateVal):
    new_value = 0
    requests_com = [0,0]
    requests_incom = [0,0]
    rewards = 0
    state_DayStart = [0,0]
    next_state = np.array([0,0])
    for i in range(11):  # request loop
        for j in range(11):
            requests_com[0] = min(i,current_state[0])
            requests_com[1] = min(j,current_state[1]) 
            if i-current_state[0] >=0:
                requests_incom[0] = i-current_state[0]
            else:
                requests_incom[0] = 0 
                
            if j-current_state[1] >=0:
                requests_incom[1] = i-current_state[1]
            else:
                requests_incom[1] = 0 
             
            rewards = ((requests_com[0] + requests_com[1]) * 10) - ((requests_incom[0] + requests_incom[1]) *2)   
            prob = cars_requested_loc1[i] * cars_requested_loc2[j]
            for k in range(11):  # return loop 
                for l in range(11):                   
                    # cars to start the day #next days start. Action is taken on current state. this is the intermediate state after action is taken 
                    state_DayStart[0] = current_state[0] - action  
                    state_DayStart[1] = current_state[1] + action 
                    # cars at end of day # next day end state. This is the final state upon which next action will be taken
                    next_state[0] = min(state_DayStart[0] - requests_com[0] + k , max_cars)
                    next_state[1] = min(state_DayStart[1] - requests_com[1] + l , max_cars)
                    prob = (cars_returned_loc1[k] * cars_returned_loc2[l])*prob
                    new_value += prob*(rewards + discount_rate*(currentStateVal[next_state[0],next_state[1]]))
        
    return new_value    

In [49]:
# policy improvement 
if improvePolicy == True:
    newPolicy = np.zeros((max_cars + 1, max_cars + 1))
    for i, j in states:
        actionReturns = []
        for action in actions:
            if ((action >= 0 and i >= action) or (action < 0 and j >= np.absolute(action))):
                actionReturns.append(new_values([i, j], action, currentStateVal))
            else:
                actionReturns.append(-float('inf'))
        bestAction = np.argmax(actionReturns)
        newPolicy[i, j] = actions[bestAction]
    policyChanges = np.sum(newPolicy != policy)
    policy = newPolicy
    improvePolicy = False

In [50]:
newPolicy

array([[ 0., -1., -2., -3., -4., -5., -5., -5., -5., -5., -4., -3., -4.,
        -4., -4., -5., -5., -5., -5., -5., -5.],
       [ 1., -1., -2., -3., -4., -5., -5., -5.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1., -5., -5., -5.],
       [ 2., -1., -2., -3., -4., -5., -5.,  2.,  2.,  2.,  2.,  2.,  2.,
         2.,  2.,  2.,  2.,  2., -5., -5., -5.],
       [ 3.,  3., -2., -3., -4., -5., -5.,  3.,  3.,  3.,  3.,  3.,  3.,
         3.,  3.,  3.,  3., -5., -5., -5., -5.],
       [ 4.,  4.,  4., -3., -4., -5.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
         4.,  4.,  4.,  4., -5., -5., -5., -5.],
       [ 5.,  5.,  5., -3., -4., -5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
         5.,  5.,  5., -4., -5., -5., -5., -5.],
       [ 5., -1., -2., -3., -4., -5., -5.,  5.,  5.,  5.,  5.,  5., -2.,
        -2., -3., -3., -4., -4., -5., -5., -5.],
       [ 5., -1., -2., -3., -4., -5., -5., -5.,  5.,  5., -1., -1., -2.,
        -2., -3., -3., -4., -4., -5., -5., -5.],
       [ 5., -1., -2., -

In [ ]:
# returns += prob * (rewards + DISCOUNT_RATE * stateValue[carsLoc1_prime, carsLoc2_prime]) 
# rewards are adjusted in order to accomodate enviornment limitations eg state is 19 so prob is
#generated for all possible returns (0,1,2,3...10) but reward is calculated on for return =  0,1 and is 0 otherwise. Hence final probability for return = 2,3 ... is zero 
# as final prob = initial_prob(returns) * reward

# first calculate value function using arbitary policy. (from each state for each possible combination of requests and returns, 
                        #calculate the total returns for a given action by calculating the possible next states and considering the values of the next states)
# then apply a grredy search method for all actions possibe from a given state. The action resukting is a state with max value is chosen for the new policy
#continue the above process for all the states and the respective possible actions 
# get new policy from above step 
# calculate the new values for each state and continue as above 